<a href="https://colab.research.google.com/github/makanadani/vetbot/blob/main/vetbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instala a ferramenta de IA Generativa do Gemini
!pip install -q google.genai

# Instala framework de agentes do Google
!pip install -q google-adk

# Instala e importa o streamlit para interface
!pip install -q streamlit

# Instala o ngrok
!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72

In [2]:
# Configura a API Key do Google Gemini
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configura o cliente da SDK do Gemini
from google import genai
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"

In [3]:
# Importa ferramentas necessárias para construção do agente
from google.colab import userdata
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings
import re # Para validar o CEP com REGEX
import gradio as gr # Para interface
warnings.filterwarnings("ignore")

In [4]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:

    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()

    # Cria uma nova sessão
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")

    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)

    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""

    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [5]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [6]:
#################################################
# --------- Agente 1: VetBot BUSCADOR --------- #
#################################################

def vetbot_buscador(sintoma, especie, idade, raca, porte, sexo, castrado, cep, data_de_hoje):
  buscador = Agent(
      name="vetbot_buscador",
      model="gemini-2.0-flash",
      description="Agente que busca informações veterinárias confiáveis no Google.",
      tools=[google_search],
      instruction="""
      Você é um assistente de pesquisa veterinária, treinado para buscar informações educativas sobre saúde e comportamento de cães e gatos no
      Google (google_search) relacionadas ao assunto abaixo, considerando a espécie, idade, raça, porte e sexo do animal. Busque por informações sobre
      o sintoma/comportamento do animal descrito, priorizando informações de fontes confiáveis e reconhecidas na medicina veterinária, como associações (WSAVA,
      AAHA, AAFCO, AVMA, FECAVA, RCVS, Fear Free Pets, CFMV, CRMV), universidades (federais e estaduais do Brasil ou de renome internacionais, como Royal College,
      Cornell, UC Davis, Colorado State University) e artigos científicos (PubVet, Scielo, Merck Veterinary Manual, DOAJ, BMC, Frontiers in Veterinary Science,
      Veterinary Sciences MDPI, NCBI PubMed Central). Tente focar em informações publicadas no último ano, sempre que possível. Busque também por hospitais
      ou clínicas veterinárias 24 horas próximos ao CEP mencionado, conferindo se estão na mesma cidade e excluindo os que não estejam.
      """
  )
  entrada_vetbot_buscador = f"""
  Por favor, pesquise sobre o seguinte:
  - Sintoma/Comportamento: {sintoma}
  - Espécie: {especie}
  - Idade: {idade}
  - Raça: {raca if raca else 'Sem raça definida'}
  - Porte: {porte}
  - Sexo: {sexo}
  - Castrado(a): {castrado}
  - CEP: {cep}
  - Data atual: {data_de_hoje}
  """
  informacoes_gerais = call_agent(buscador, entrada_vetbot_buscador)
  return informacoes_gerais

In [7]:
#################################################
# -------- Agente 2: VetBot FILTRADOR -------- #
#################################################

def vetbot_filtrador(sintoma, especie, idade, raca, porte, sexo, castrado, cep, informacoes_buscadas):
    filtrador = Agent(
        name="vetbot_filtrador",
        model="gemini-2.0-flash",
        description="Agente que filtra, valida e sintetiza informações veterinárias com base científica.",
        tools=[google_search],
        instruction="""
        Você é um assistente científico e veterinário, especializado em estudos de cães e gatos. Sua tarefa é analisar as informações encontradas sobre
        o sintoma/comportamento no animal cujos dados constam na entrada. Com base na lista de informações e fontes fornecidas, sintetize os principais
        pontos de forma clara e didática para um tutor. Priorize informações com aparente base científica ou clínica, descartando explicitamente ou
        tratando com extrema cautela informações de blogs pessoais, fóruns e sites sem fontes científicas claras ou que pareçam anedóticas. Foque no
        conhecimento embasado sobre as possíveis causas gerais, o que observar em casa, e os níveis de urgência relevantes para um animal da espécie,
        idade, raça, porte e sexo apresentados. Mantenha um tom informativo e educativo, removendo sugestões de diagnósticos e de tratamentos. Liste as
        fontes das informações e os hospitais e clínicas veterinários 24 horas recebidos.
        """
    )
    entrada_vetbot_filtrador = f"""
    Detalhes do Animal:
    - Espécie: {especie}
    - Idade: {idade}
    - Raça: {raca if raca else 'Sem raça definida'}
    - Porte: {porte}
    - Sexo: {sexo if sexo else 'Não informado'}
    - Castrado(a): {castrado}
    - CEP: {cep}
    - Sintoma/Comportamento principal: {sintoma}
    A seguir, estão as informações encontradas na pesquisa inicial.
    Por favor, analise-as, filtre o conteúdo confiável e sintetize os pontos principais relevantes para este animal: {informacoes_buscadas}
    """
    informacoes_filtradas = call_agent(filtrador, entrada_vetbot_filtrador)
    return informacoes_filtradas

In [22]:
##########################################
# ------ Agente 3: VetBot REVISOR ------ #
##########################################
def vetbot_revisor(nome_pet, sintoma, especie, idade, raca, porte, sexo, castrado, cep, resposta_gerada):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        description="Agente revisor final para garantir segurança, clareza e relevância veterinária.",
        instruction="""
        Você é um médico veterinário experiente, especializado em atendimento de cães e gatos. Sua função é revisar o texto gerado para o tutor do animal
        descrito. Como veterinário, você sabe a importância de não realizar prescrições, diagnósticos ou sugerir tratamentos sem uma consulta presencial e
        exames. Revise o texto fornecido (o "Rascunho") para:
        1. Exclua quaisquer informações que possam configurar prescrição, diagnóstico ou orientação sobre tratamentos específicos (nomes de remédios,
        doses, procedimentos, o que fazer, etc.).
        2. Mantenha apenas o que for relevante para um tutor entender o sintoma ou comportamento apresentado no contexto da espécie, idade, raça, porte e
        sexo do animal. Isso inclui informações sobre o comportamento normal da espécie, possíveis causas gerais, o que observar em casa e o que não fazer,
        desmistificando medidas não científicas e citando as fontes das informações. Use uma linguagem clara, empática e acessível ao tutor do animal.
        3. Obrigatório: Liste o nível de urgência aparente dentre a lista: EMERGÊNCIA, CONSULTA, MONITORAMENTO e COMPORTAMENTO COMUM.
        4. Obrigatório: Personalize a resposta utilizando o nome real do pet e levando em consideração seus dados nas informações fornecidas.
        5. Obrigatório: Reforce claramente em sua resposta final (NO INÍCIO E NO FIM) que você é apenas um assistente virtual e não substitui
        o atendimento médico veterinário especializado.
        6. Obrigatório: Liste os hospitais ou clínicas veterinários 24 horas recebidos.
        7. Quando necessário, oriente o tutor sobre os riscos de não buscar esta consulta, considerando a possível gravidade implícita do sintoma
        para um animal com as características apresentadas. Use as categorias de nível de urgência de forma clara.
        """
    )
    entrada_vetbot_revisor = f"""
    Detalhes do Animal:
    - Nome do Pet: {nome_pet}
    - Espécie: {especie}
    - Idade: {idade}
    - Raça: {raca if raca else 'Sem raça definida'}
    - Porte: {porte}
    - Sexo: {sexo if sexo else 'Não informado'}
    - Castrado(a): {castrado}
    - CEP: {cep}
    - Sintoma/Comportamento principal: {sintoma}
    Rascunho da resposta a ser revisada:
    {resposta_gerada}
    """

    resposta_final = call_agent(revisor, entrada_vetbot_revisor)

    markdown_resposta = f"""\
    {resposta_final}
    """

    return markdown_resposta


In [23]:
# Interface com Gradio
def vetbot_pipeline(nome_pet, especie, idade, raca, porte, sexo, castrado, cep, sintoma):
    nome_pet = nome_pet.strip().capitalize()
    raca = raca.strip().capitalize() if raca else "Sem raça definida"
    cep_padrao = re.compile(r'^\d{5}-?\d{3}$')
    data_de_hoje = date.today().strftime("%d/%m/%Y")

    if not nome_pet:
        yield "❌ Por favor, digite o nome do seu pet.", ""
        return
    if not sintoma.strip():
        yield "❌ Por favor, descreva o sintoma ou o comportamento observado.", ""
        return
    if cep and not cep_padrao.fullmatch(cep.strip()):
        yield "❌ CEP inválido.", ""
        return
    cep = cep.replace("-", "") if cep else ""

    if especie.lower() == "gato":
        porte = "não aplicável"

    try:
        yield "", "🔎 Etapa 1: Buscando informações sobre o sintoma..."
        informacoes_buscadas = vetbot_buscador(sintoma, especie, idade, raca, porte, sexo, castrado, cep, data_de_hoje)

        yield "", "🧪 Etapa 2: Filtrando resultados com base nas características do pet..."
        informacoes_filtradas = vetbot_filtrador(sintoma, especie, idade, raca, porte, sexo, castrado, cep, informacoes_buscadas)

        yield "", "🧠 Etapa 3: Gerando resposta personalizada com base nas evidências..."
        resposta = vetbot_revisor(nome_pet, sintoma, especie, idade, raca, porte, sexo, castrado, cep, informacoes_filtradas)

        yield resposta, "✅ Pronto! Aqui está a resposta completa."

    except Exception as e:
        yield f"❌ Erro ao processar: {e}", "⚠️ Ocorreu uma falha na execução. Tente novamente."


def atualizar_visibilidade_pelo_animal(valor_especie):
    if valor_especie.lower() == "cão":
        return gr.update(visible=True), None
    else:
        return gr.update(visible=False), None


with gr.Blocks(title="VetBot 🐾") as demo:
    gr.Image(
        value="vetbot.png",
        show_label=False,
        width=500
    )

    gr.Markdown("<h1 style='text-align: center;'>🐶🤖 VetBot - O Assistente Virtual do Seu Pet! 🐱</h1>")
    gr.Markdown("<p style='text-align: center;'>Obtenha orientações educativas sobre sintomas ou comportamentos do seu pet!</p>")
    gr.Markdown("<h3 style='text-align: center;'><strong>O VetBot NÃO SUBSTITUI O ATENDIMENTO VETERINÁRIO!</strong></h3>")

    with gr.Row():
        nome_pet = gr.Textbox(label="📝 Nome do Pet*")
        especie = gr.Dropdown(["Cão", "Gato"], label="🐾 Espécie*", interactive=True)

    idade = gr.Dropdown([
        "Filhote (até 6 meses)",
        "Jovem (de 7 a 18 meses)",
        "Adulto (de 19 meses a 7 anos)",
        "Meia-idade (de 7 a 10 anos)",
        "Idoso (acima de 10 anos)",
        "Não sei"
    ], label="🎂 Faixa Etária*")

    raca = gr.Textbox(label="🐕🐈 Raça (opcional)", placeholder="Ex: Labrador, Siamês")

    porte = gr.Dropdown([
        "Pequeno (até 10 kg)",
        "Médio (11-25 kg)",
        "Grande (26-44 kg)",
        "Gigante (mais de 45 kg)",
        "Não sei"],
        label="📏 Porte*",
        interactive=True,
        value=None,
        visible=True
    )

    sexo = gr.Dropdown(["Macho", "Fêmea", "Não sei"], label="⚧️ Sexo*")
    castrado = gr.Dropdown(["Sim", "Não", "Não sei"], label="✂️ Castrado(a)?*")
    cep = gr.Textbox(label="📍 CEP (opcional)", placeholder="Ex: 12345000 ou 12345-000")
    sintoma = gr.Textbox(label="🩺 Descreva o sintoma ou comportamento observado:*", lines=3, placeholder="Ex: Está vomitando há 1 dia.")

    botao = gr.Button("🔎 Consultar VetBot")
    saida = gr.Markdown(label="📋 Resposta do VetBot")
    status_output = gr.Markdown(label="📡 Status do VetBot", value="")

    especie.change(fn=atualizar_visibilidade_pelo_animal, inputs=especie, outputs=[porte, porte])

    botao.click(
        fn=vetbot_pipeline,
        inputs=[nome_pet, especie, idade, raca, porte, sexo, castrado, cep, sintoma],
        outputs=[saida, status_output]
    )


demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3c05662fa12f728262.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
